In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [29]:
#!pip install statsmodels

In [30]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
# import lazypredict

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

import statsmodels.tsa.stattools as st
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA

In [31]:
df = pd.read_csv('Data/GCB2022v27_MtCO2_flat.csv')
df

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Afghanistan,AFG,1750,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1751,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1752,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1753,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1754,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
63099,Global,WLD,2017,36096.739276,14506.973805,12242.627935,7144.928128,1507.923185,391.992176,302.294047,4.749682
63100,Global,WLD,2018,36826.506600,14746.830688,12266.016285,7529.846784,1569.218392,412.115746,302.478706,4.792753
63101,Global,WLD,2019,37082.558969,14725.978025,12345.653374,7647.528220,1617.506786,439.253991,306.638573,4.775633
63102,Global,WLD,2020,35264.085734,14174.564010,11191.808551,7556.290283,1637.537532,407.583673,296.301685,4.497423


In [32]:
# dropping least important columns
df = df.drop(['Cement','Flaring', 'Other','Per Capita'], axis=1)

In [33]:
#dropping years less than 1850

df.drop(df[df['Year'] < 1980].index, inplace = True) # the version used for germany, japan, global, senegal. Not working for USA, china, india
#df.drop(df[df['Year'] < 1920].index, inplace = True)

In [34]:
df = df.reset_index()

In [35]:
df.columns

Index(['index', 'Country', 'ISO 3166-1 alpha-3', 'Year', 'Total', 'Coal',
       'Oil', 'Gas'],
      dtype='object')

In [36]:
## Filling nans with zero for the inpuut features
df.Coal = df['Coal'].fillna(0, inplace = False)
df.Oil = df['Oil'].fillna(0, inplace = False)
df.Gas = df['Gas'].fillna(0, inplace = False)
#df.Cement = df['Cement'].fillna(0, inplace = False)
#df.Flaring = df['Flaring'].fillna(0, inplace = False)
#df.Other = df['Other'].fillna(0, inplace = False)
#df['Per Capita'] = df['Per Capita'].fillna(0, inplace = False)

In [37]:
# Checking the count of null values
df.isna().sum()

index                   0
Country                 0
ISO 3166-1 alpha-3    252
Year                    0
Total                   0
Coal                    0
Oil                     0
Gas                     0
dtype: int64

In [38]:
df.reset_index()
df

,index,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas
0,230,Afghanistan,AFG,1980,1.756302,0.315762,0.925256,0.187254
1,231,Afghanistan,AFG,1981,1.978463,0.333424,1.014928,0.304112
2,232,Afghanistan,AFG,1982,2.094581,0.384720,0.992944,0.395712
3,233,Afghanistan,AFG,1983,2.519954,0.384720,1.220112,0.615552
4,234,Afghanistan,AFG,1984,2.821540,0.392556,1.133644,0.931863
...,...,...,...,...,...,...,...,...
9739,63099,Global,WLD,2017,36096.739276,14506.973805,12242.627935,7144.928128
9740,63100,Global,WLD,2018,36826.506600,14746.830688,12266.016285,7529.846784
9741,63101,Global,WLD,2019,37082.558969,14725.978025,12345.653374,7647.528220
9742,63102,Global,WLD,2020,35264.085734,14174.564010,11191.808551,7556.290283


In [39]:
df['Country'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bonaire, Saint Eustatius and Saba', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cambodia', 'Canada', 'Cape Verde', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Christmas Island', 'Colombia',
       'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire",
       'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic',
       'North Korea', 'Democratic Republic of the Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
   

List of countries: USA, Japan, Germany, China, India, Brazil, Senegal, Cambodia, Congo (9)

The whole world: Global


In [40]:
# USA data set
df_usa = df.loc[df['Country'] == 'USA']
df_usa = df_usa.reset_index().drop(columns='index').drop(columns='level_0')
df_usa

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas
0,USA,USA,1980,4808.520356,1453.794000,2244.444000,1058.054000
1,USA,USA,1981,4686.369198,1502.932000,2101.611000,1032.089000
2,USA,USA,1982,4447.255938,1449.085000,1994.450000,960.901000
3,USA,USA,1983,4429.374051,1504.796000,1978.053000,901.094000
4,USA,USA,1984,4662.141491,1616.103000,2034.520000,962.252000
5,USA,USA,1985,4652.579989,1655.025000,2023.815000,926.516000
6,USA,USA,1986,4663.369645,1635.733000,2112.065000,867.866000
7,USA,USA,1987,4825.650651,1709.484000,2143.098000,922.972000
8,USA,USA,1988,5050.475381,1792.727000,2241.115000,964.373000
9,USA,USA,1989,5132.134279,1813.144000,2239.733000,1026.955000


In [41]:
# Japan data set
df_jap = df.loc[df['Country'] == 'Japan']
df_jap = df_jap.reset_index().drop(columns='index').drop(columns='level_0')
df_jap

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas
0,Japan,JPN,1980,944.868566,224.299088,630.658672,47.932448
1,Japan,JPN,1981,926.835495,247.357616,590.551194,48.592160
2,Japan,JPN,1982,897.407791,242.630080,566.787824,49.746128
3,Japan,JPN,1983,880.909402,229.795088,559.573408,53.347840
4,Japan,JPN,1984,937.069892,260.175961,568.097320,71.696872
5,Japan,JPN,1985,912.400813,271.788192,529.480976,76.980640
6,Japan,JPN,1986,912.259933,257.465616,540.355728,81.252864
7,Japan,JPN,1987,902.530350,250.394096,536.237392,82.707472
8,Japan,JPN,1988,985.464274,278.511632,584.909968,86.206592
9,Japan,JPN,1989,1021.681739,277.104656,615.156288,92.724848


In [42]:
# Germany data set
df_ger = df.loc[df['Country'] == 'Germany']
df_ger = df_ger.reset_index().drop(columns='index').drop(columns='level_0')
df_ger

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas
0,Germany,DEU,1980,1100.065995,551.175520,412.456480,115.452640
1,Germany,DEU,1981,1048.523357,557.748736,361.299712,110.000608
2,Germany,DEU,1982,1015.733894,550.812784,344.463632,102.016752
3,Germany,DEU,1983,1011.604905,553.738320,333.397148,106.021121
4,Germany,DEU,1984,1033.244579,572.331456,334.889600,108.531344
5,Germany,DEU,1985,1044.117955,579.375692,340.221911,108.531724
6,Germany,DEU,1986,1047.471904,573.762079,356.828387,100.550801
7,Germany,DEU,1987,1032.442842,560.223918,345.534743,109.546660
8,Germany,DEU,1988,1029.024130,552.860960,351.260352,107.637328
9,Germany,DEU,1989,1008.792342,546.778720,322.794736,121.908608


For developing and underdeveloped countries, we start from 1950 as there was no entry before that

In [43]:
# China data set
df_chi = df.loc[df['Country'] == 'China']
df_chi = df_chi.reset_index().drop(columns='index').drop(columns='level_0')

df_chi.drop(df_chi[df_chi['Year'] < 1950].index, inplace = True)

df_chi

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas
0,China,CHN,1980,1494.495884,1122.664256,273.202496,27.318784
1,China,CHN,1981,1476.487854,1125.004730,256.374392,24.387646
2,China,CHN,1982,1606.585013,1248.856080,257.810032,22.900000
3,China,CHN,1983,1694.216811,1321.842960,263.771360,23.438608
4,China,CHN,1984,1844.828012,1454.197632,271.451104,24.226368
5,China,CHN,1985,1998.082904,1584.511456,284.593872,24.819936
6,China,CHN,1986,2104.213354,1657.391354,303.068297,26.351535
7,China,CHN,1987,2257.738539,1770.268928,321.523328,26.600640
8,China,CHN,1988,2425.893553,1892.697824,343.129936,27.311456
9,China,CHN,1989,2463.654367,1920.239372,354.441244,28.821068


In [44]:
# India data set
df_ind = df.loc[df['Country'] == 'India']
df_ind = df_ind.reset_index().drop(columns='index').drop(columns='level_0')

df_ind.drop(df_ind[df_ind['Year'] < 1950].index, inplace = True)

df_ind

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas
0,India,IND,1980,291.711364,197.337053,83.176464,2.487856
1,India,IND,1981,314.966469,210.305446,89.713040,2.883568
2,India,IND,1982,325.380295,213.123221,94.406624,4.360160
3,India,IND,1983,352.199157,232.519456,99.169824,5.243184
4,India,IND,1984,361.559743,231.492925,106.104774,6.170118
5,India,IND,1985,397.592455,253.289252,116.759685,7.199698
6,India,IND,1986,426.309754,273.171658,121.857312,10.310496
7,India,IND,1987,455.343444,291.856287,127.671122,12.776272
8,India,IND,1988,491.692596,317.172040,133.615088,15.205600
9,India,IND,1989,540.651493,339.625896,150.714021,18.517739


In [45]:
# Brazil data set
df_bra = df.loc[df['Country'] == 'Brazil']
df_bra = df_bra.reset_index().drop(columns='index').drop(columns='level_0')

df_bra.drop(df_bra[df_bra['Year'] < 1950].index, inplace = True)

df_bra

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas
0,Brazil,BRA,1980,185.471117,19.986728,150.210064,1.883259
1,Brazil,BRA,1981,170.613975,19.723312,135.527696,1.685440
2,Brazil,BRA,1982,170.811520,21.551648,133.193728,2.381600
3,Brazil,BRA,1983,165.348839,23.926447,125.740255,3.378282
4,Brazil,BRA,1984,167.076404,28.996896,122.062496,4.169632
5,Brazil,BRA,1985,179.589112,34.870994,127.517108,4.994133
6,Brazil,BRA,1986,196.613291,35.123104,142.375712,5.986976
7,Brazil,BRA,1987,205.332593,40.234384,145.541408,6.639168
8,Brazil,BRA,1988,207.069970,39.776384,147.651872,6.840688
9,Brazil,BRA,1989,211.444880,39.930272,151.579680,7.306016


In [46]:
# Senegal data set
df_sen = df.loc[df['Country'] == 'Senegal']
df_sen = df_sen.reset_index().drop(columns='index').drop(columns='level_0')

df_sen.drop(df_sen[df_sen['Year'] < 1950].index, inplace = True)

df_sen

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas
0,Senegal,SEN,1980,3.336441,0.000000,3.154704,0.000000
1,Senegal,SEN,1981,3.279857,0.000000,3.107072,0.000000
2,Senegal,SEN,1982,3.036573,0.000000,2.872576,0.000000
3,Senegal,SEN,1983,2.611411,0.000000,2.432896,0.000000
4,Senegal,SEN,1984,3.284168,0.000000,3.114400,0.000000
5,Senegal,SEN,1985,2.650504,0.000000,2.473200,0.000000
6,Senegal,SEN,1986,2.636446,0.000000,2.480280,0.000000
7,Senegal,SEN,1987,2.424574,0.000000,2.264352,0.000000
8,Senegal,SEN,1988,2.710809,0.000000,2.531824,0.014656
9,Senegal,SEN,1989,3.661877,0.000000,3.488128,0.014656


In [47]:
# Cambodia data set
df_cam = df.loc[df['Country'] == 'Cambodia']
df_cam = df_cam.reset_index().drop(columns='index').drop(columns='level_0')

df_cam.drop(df_cam[df_cam['Year'] < 1950].index, inplace = True)

df_cam

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas
0,Cambodia,KHM,1980,0.285792,0.000000,0.285792,0.0
1,Cambodia,KHM,1981,0.300448,0.000000,0.300448,0.0
2,Cambodia,KHM,1982,0.337088,0.000000,0.337088,0.0
3,Cambodia,KHM,1983,0.366400,0.000000,0.366400,0.0
4,Cambodia,KHM,1984,0.410368,0.000000,0.410368,0.0
5,Cambodia,KHM,1985,0.417696,0.000000,0.417696,0.0
6,Cambodia,KHM,1986,0.432352,0.000000,0.432352,0.0
7,Cambodia,KHM,1987,0.436016,0.000000,0.436016,0.0
8,Cambodia,KHM,1988,0.450672,0.000000,0.450672,0.0
9,Cambodia,KHM,1989,0.450672,0.000000,0.450672,0.0


In [48]:
# Congo data set
df_con = df.loc[df['Country'] == 'Congo']
df_con = df_con.reset_index().drop(columns='index').drop(columns='level_0')

df_con.drop(df_con[df_con['Year'] < 1950].index, inplace = True)

df_con

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas
0,Congo,COG,1980,0.405529,0.0,0.274800,0.003664
1,Congo,COG,1981,0.470724,0.0,0.318768,0.000000
2,Congo,COG,1982,1.343102,0.0,1.168816,0.003664
3,Congo,COG,1983,1.142452,0.0,0.970960,0.003664
4,Congo,COG,1984,1.139504,0.0,0.970960,0.003664
5,Congo,COG,1985,1.275224,0.0,1.077792,0.007307
6,Congo,COG,1986,1.062373,0.0,0.864704,0.003664
7,Congo,COG,1987,1.320092,0.0,1.132176,0.003664
8,Congo,COG,1988,1.492941,0.0,1.289728,0.003664
9,Congo,COG,1989,1.495627,0.0,1.271408,0.003664


In [49]:
# Global data set
df_glob = df.loc[df['Country'] == 'Global']
df_glob = df_glob.reset_index().drop(columns='index').drop(columns='level_0')
df_glob

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas
0,Global,WLD,1980,19501.263339,7024.128811,8951.775558,2758.101427
1,Global,WLD,1981,19036.118109,7004.883319,8550.304967,2794.346721
2,Global,WLD,1982,18884.487014,7126.512351,8345.175954,2731.273279
3,Global,WLD,1983,19006.539624,7314.190173,8260.815187,2760.541893
4,Global,WLD,1984,19664.753809,7668.797447,8328.840383,3010.425483
5,Global,WLD,1985,20325.486795,8217.823510,8326.771194,3121.400874
6,Global,WLD,1986,20627.358491,8335.814386,8576.333790,3045.544194
7,Global,WLD,1987,21273.904184,8617.360981,8654.658723,3305.280002
8,Global,WLD,1988,22107.794982,8910.819131,8972.672132,3466.022378
9,Global,WLD,1989,22410.401527,8909.615731,9130.079809,3639.817109


In [50]:
#  plot to see total emission versus year for 'global'
fig_glob = px.line(df_glob, x="Year", y="Total")
fig_glob.update_xaxes(rangeslider_visible=True)
fig_glob.show()

In [51]:
#  plot to see total emission versus year for 'Senegal'
fig_sen = px.line(df_sen, x="Year", y="Total")
fig_sen.update_xaxes(rangeslider_visible=True)
fig_sen.show()

In [52]:
#  plot to see total emission versus year for 'USA'
fig_usa = px.line(df_usa, x="Year", y="Total")
fig_usa.update_xaxes(rangeslider_visible=True)
fig_usa.show()

In [53]:
#  plot to see total emission versus year for 'Germany'
fig_ger = px.line(df_ger, x="Year", y="Total")
fig_ger.update_xaxes(rangeslider_visible=True)
fig_ger.show()

In [54]:
# USA, Japan, Germany, China, India, Brazil, Senegal, Cambodia, Congo (9)
df_usa.to_csv('Data/usa.csv', index=False)
df_jap.to_csv('Data/japan.csv', index=False)
df_ger.to_csv('Data/Germany.csv', index=False)
df_chi.to_csv('Data/China.csv', index=False)
df_ind.to_csv('Data/India.csv', index=False)
df_bra.to_csv('Data/Brazil.csv', index=False)
df_sen.to_csv('Data/Senegal.csv', index=False)
df_cam.to_csv('Data/Cambodia.csv', index=False)
df_con.to_csv('Data/Congo.csv', index=False)
df_glob.to_csv('Data/Global.csv', index=False)
df.to_csv('Data/Cleaned_whole_data.csv', index=False)
